In [1]:
####### importing all required packages here ##########
import numpy as np
import random
import time
import matplotlib.pyplot as plt
from qiskit import *
from qiskit.optimization.applications.ising import max_cut
from qiskit.optimization.converters import IsingToQuadraticProgram
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.aqua import QuantumInstance
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.aqua.algorithms import QAOA

In [2]:
###### Fucntion for creating random adjacency matrix. #########

def create_adjacency_matrix(number_of_vertices, probability):
    adj_matrix = np.ones((number_of_vertices,number_of_vertices))
    for i in range(0,number_of_vertices):
        for j in range(0,number_of_vertices):
            if random.random()>probability:
                adj_matrix[i,j]=0

    return adj_matrix

In [3]:
###### Function for converting binary number to decimal #######

def dec_to_bin(decimal, no_bits):
    binary = np.zeros((no_bits))
    for i in range(no_bits-1,-1,-1):
        #print(decimal, i)
        if int(decimal/(2**i)):
            binary[i]=1
        decimal = decimal%(2**i)
    
    return binary

In [4]:
####### Using Brute force ########

def brute_force(n, adj_matrix):
    best_cost = 0
    best_case = None
    for k in range(2**n):
        binary_k=dec_to_bin(k, n)
        temp_cost = 0
        for i in range(n):
            for j in range(n):
                temp_cost = temp_cost + adj_matrix[i,j]*binary_k[i]*(1-binary_k[j])
        if temp_cost > best_cost:
            best_cost = temp_cost
            best_case = binary_k

    return best_cost

In [5]:
######## function for getting n dependant cost and time using brute force ##########
def get_cost_and_time_vs_n(n1, n2):
    cost_vs_n = list()
    time_vs_n = list()
    cost = list()
    tim = list()
    for j in range(n1,n2):
        for i in range(5):
            initial_time = time.time()
            cost.append(brute_force(j, create_adjacency_matrix(j, 0.5)))
            final_time = time.time()
            tim.append(final_time - initial_time)
        average_cost = sum(cost)/5
        average_time = sum(tim)/5 
        (x, y) = (j, average_cost)
        (x, z) = (j, average_time)
        cost_vs_n.append((x, y))
        time_vs_n.append((x, z))
        print("n=", x, "cost=", y, "time=", z)
        cost.clear()
        tim.clear()
    return cost_vs_n, time_vs_n

In [6]:
######## Function for getting cost using numpyeigensolver ############
def using_numpyeigensolver(n, adj_matrix):
    qubitop, offset = max_cut.get_operator(adj_matrix)
    qudratic = IsingToQuadraticProgram()
    qudratic_problem = qudratic.encode(qubitop, offset)
    solver = MinimumEigenOptimizer(NumPyMinimumEigensolver())
    result = solver.solve(qudratic_problem)
    best_cost = float(str(result).split('-')[1])
    return best_cost


In [7]:
########  function for getting n dependant cost and time using Numpyeigensolver #########
def get_cost_and_time_numpyeigensolver(n1, n2):
    cost_vs_n_fromeigensolver = list()
    time_vs_n_fromeigensolver = list()
    cost = list()
    tim = list()
    for j in range(n1,n2):
        for i in range(5):
            initial_time = time.time()
            cost.append(using_numpyeigensolver(j, create_adjacency_matrix(j, 0.5)))
            final_time = time.time()
            tim.append(final_time - initial_time)
        average_cost = sum(cost)/5
        average_time = sum(tim)/5 
        (x, y) = (j, average_cost)
        (x, z) = (j, average_time)
        cost_vs_n_fromeigensolver.append((x, y))
        time_vs_n_fromeigensolver.append((x, z))
        print("n=", x, "cost=",  y, "time=",  z)
        cost.clear()
        tim.clear()
    return cost_vs_n_fromeigensolver, time_vs_n_fromeigensolver

In [9]:
def using_vqe(n, adj_matrix):
    qubitop, offset = max_cut.get_operator(adj_matrix)
    qudratic = IsingToQuadraticProgram()
    qudratic_problem = qudratic.encode(qubitop, offset)
    seed = 10598
    backend = Aer.get_backend('qasm_simulator')
    quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)
    spsa = SPSA(max_trials=300)
    ry = TwoLocal(qubitop.num_qubits, 'ry', 'cz', reps=5, entanglement='linear')
    vqe = VQE(qubitop, ry, spsa, quantum_instance=quantum_instance)
    #results = vqe.run(quantum_instance)
    #best_time = result.optimizer_time
    vqe_optimizer = MinimumEigenOptimizer(vqe)
    result = vqe_optimizer.solve(qudratic_problem)
    best_cost =float(str(result).split('-')[1])
    return best_cost

using_vqe(9, create_adjacency_matrix(9, 0.5))

13.0

In [ ]:
########  function for getting n dependant cost and time using VQE #########
def get_cost_and_time_VQE(n1, n2):
    cost_vs_n_fromVQE = list()
    time_vs_n_fromVQE = list()
    cost = list()
    tim = list()
    for j in range(n1,n2):
        for i in range(5):
            initial_time = time.time()
            cost.append(using_vqe(j, create_adjacency_matrix(j, 0.5)))
            final_time = time.time()
            tim.append(final_time - initial_time)
        average_cost = sum(cost)/5
        average_time = sum(tim)/1500 
        (x, y) = (j, average_cost)
        (x, z) = (j, average_time)
        cost_vs_n_fromVQE.append((x, y))
        time_vs_n_fromVQE.append((x, z))
        print("n=", x, "cost=", y, "time=", z)
        cost.clear()
        tim.clear()
    return cost_vs_n_fromVQE, time_vs_n_fromVQE

In [14]:
def cost_qaoa(n, adj_matrix):
    operator, offset = max_cut.get_operator(adj_matrix)
    seed = 10598
    backend = Aer.get_backend('qasm_simulator')
    quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)
    spsa = SPSA(max_trials=300)
    qaoa_API = QAOA(operator, spsa, p = 4, max_evals_grouped = 1 ,quantum_instance = quantum_instance )
    result = qaoa_API.run(quantum_instance)
    print(QAOA.get_optimal_cost(qaoa_API)+offset)
    print(QAOA.get_optimal_vector(qaoa_API))

cost_qaoa(5, create_adjacency_matrix(5, 0.5))

-3.8408203125
{'01100': 1, '11111': 1, '00011': 231, '10010': 17, '10001': 20, '00111': 216, '11100': 219, '01110': 22, '10111': 1, '00100': 1, '00101': 1, '11011': 1, '01101': 19, '00000': 4, '10110': 9, '11000': 217, '01010': 18, '01001': 13, '10101': 13}


In [ ]:
######## getting cost and time from every method #########

print("Cost and time using Brute Force")
cost_vs_n, time_vs_n = get_cost_and_time_vs_n(4, 10)

print("")

print("Cost and time using NumpyEigenSolver")
cost_vs_n_fromeigensolver, time_vs_n_fromeigensolver = get_cost_and_time_numpyeigensolver(4, 10)

print("")

print("Cost and time using VQE")
cost_vs_n_fromVQE, time_vs_n_fromVQE = get_cost_and_time_VQE(4, 10)

In [ ]:
#### plotting cost vs n #####
plt.title("Cost Vs Number of vertices")
plt.ylabel('Cost')
plt.xlabel('number of vertices')
x, y = zip(*cost_vs_n)
plt.plot(x, y, label = 'Brute Force')
x, y = zip(*cost_vs_n_fromeigensolver)
plt.plot(x, y, label = 'Numpyeigensolver')
x, y = zip(*cost_vs_n_fromVQE)
plt.plot(x, y, label = 'VQE')
plt.legend()


In [ ]:
######### Plotting Time Vs n ########
plt.title("Time Vs Number of vertices")
plt.ylabel('Time')
plt.xlabel('number of vertices')
x, y = zip(*time_vs_n)
plt.plot(x, y, label = 'Brute Force')
x, y = zip(*time_vs_n_fromeigensolver)
plt.plot(x, y, label = 'Numpyeigensolver')
x, y = zip(*time_vs_n_fromVQE)
plt.plot(x, y, label = 'VQE')
plt.legend()